# Encoding protein sequences with one-hot and AAindex encoding

This notebook shows how to encode protein sequences with a combination of one-hot and AAindex encoding.
The resulting encoded sequences are used as input to the neural networks.

In [1]:
# reload modules before executing code in order to make development and debugging easier
%load_ext autoreload
%autoreload 2

In [1]:
# this jupyter notebook is running inside of the "notebooks" directory
# for relative paths to work properly, we need to set the current working directory to the root of the project
# for imports to work properly, we need to add the code folder to the system path
import os
from os.path import abspath, join, isdir
import sys
if not isdir("notebooks"):
    # if there's a "notebooks" directory in the cwd, we've already set the cwd so no need to do it again
    os.chdir("..")
module_path = abspath("code")
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import numpy as np
import constants
import encode as enc

In [3]:
pca = np.load("data/aaindex/pca-19.npy")
pca

array([[-1.20350152e-01,  5.42581367e+00,  1.70167446e+01,
         2.20421433e+00,  2.81965852e+00,  3.03139782e+00,
        -3.77325773e+00, -4.51975614e-01,  3.96887374e+00,
        -1.78279221e+00,  8.02974701e-01,  1.69522071e+00,
         2.01125598e+00,  1.66262484e+00,  5.32224655e+00,
         4.57231569e+00,  1.97126663e+00, -2.26111293e+00,
         6.63553357e-01],
       [ 8.22706604e+00,  8.15196609e+00, -6.01552105e+00,
        -1.43637590e+01,  1.24458961e+01, -7.31303787e+00,
        -3.36787295e+00, -5.20933151e+00, -1.91490364e+00,
         4.09993124e+00,  2.07780027e+00,  8.94995213e-01,
         1.29136181e+00, -3.93140763e-01, -7.04672337e-01,
         6.00543201e-01,  4.52601723e-02, -6.93937361e-01,
         5.89501560e-01],
       [-1.85021572e+01, -3.04121518e+00, -7.49196038e-02,
        -1.68545008e+00,  5.73821878e+00,  5.06379223e+00,
         9.41339207e+00, -2.16880608e+00, -1.97001410e+00,
        -1.62866640e+00,  1.51652515e+00, -1.24244821e+00,
    

The main function for encoding variants is `enc.encode()`. It supports encoding variants specified in two forms: as a full sequence of amino acids or as a list of mutations in the form &lt;current_aa&gt;&lt;position&gt;&lt;replacement_aa&gt;.

# Encode a full sequence
Encoding a variant specified as a full sequence is straightforward. Let's say you want to encode a GB1 variant where the amino acid in position 3 (0-indexed) has been mutated from a K to an A. The full sequence of this variant is:

> "MQYALILNGKTLKGETTTEAVDAATAEKVFKQYANDNGVDGEWTYDDATKTFTVTE"

In [4]:
char_seq = "MQYALILNGKTLKGETTTEAVDAATAEKVFKQYANDNGVDGEWTYDDATKTFTVTE"
K3A = enc.encode(encoding="one_hot,aa_index", char_seqs=char_seq)
K3A

array([[ 0.        ,  0.        ,  0.        , ..., -3.4104924 ,
         0.07120343, -0.9318722 ],
       [ 0.        ,  0.        ,  0.        , ...,  6.655027  ,
         0.6539753 , -0.17933992],
       [ 0.        ,  0.        ,  0.        , ..., -0.842466  ,
        -1.7963753 ,  0.29463208],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.2865151 ,
         2.292554  , -5.5230074 ],
       [ 0.        ,  0.        ,  0.        , ..., -1.9239038 ,
        -4.5834727 ,  0.11366256],
       [ 0.        ,  0.        ,  0.        , ..., -4.8518243 ,
         0.7368123 ,  0.46764046]], dtype=float32)

## Encode multiple sequences

You can also specify a list of sequences to encode.

In [5]:
char_seqs = ["MQYALILNGKTLKGETTTEAVDAATAEKVFKQYANDNGVDGEWTYDDATKTFTVTE",
             "MLYALILNGKTLKGETTTEAVDAATAEKVFKQYANDNGVDGEWTYDDATKTFTVTE"]
encoded_seqs = enc.encode(encoding="one_hot,aa_index", char_seqs=char_seqs)
encoded_seqs.shape

(2, 56, 40)

Dimension 1 corresponds to the number of variants, dimension 2 corresponds to the number of amino acids, and dimension 3 corresponds to the number of features for each amino acid (21 for one-hot encoding and 19 for AAindex for a total of 40).

## Use a different encoding 
If you want to test only one-hot or only AAindex encoding, just specify the single encoding in the function call.

In [6]:
char_seq = "MLYKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGVDGEWTYDDATKTFTVTE"
Q1L = enc.encode(encoding="one_hot", char_seqs=char_seq)
display(Q1L.shape)
display(Q1L)

(56, 21)

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False,  True],
       ...,
       [False, False, False, ...,  True, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

# Encode lists of mutations
A more convenient way to specify variants is as a list of mutations in the form &lt;current_aa&gt;&lt;position&gt;&lt;replacement_aa&gt;. The variant from the previous section where the amino acid in position 3 (0-indexed) has been mutated from a K to an A can be simply specified as

> "K3A"

Multiple mutations in a single variant can be separated by commas:

> "Q1L,K3A"

If you are specifying a list of mutations, you must also specify the wild type sequence `wt_aa` and offset `wt_offset`. The offset is needed if your sequence has a non-zero starting point. For example, if your sequence starts at position 126, and you have variants starting with "M126", you must specify an offset of 126. This helps maintain 0-based indexing in the internal code.

In [ ]:
variants = ["K3A", "Q1L,K3A"]  # encode two variants
gb1_wt = "MQYKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGVDGEWTYDDATKTFTVTE"
gb1_offset = 0
encoded_variants = enc.encode(encoding="one_hot,aa_index", variants=variants, wt_aa=gb1_wt, wt_offset=gb1_offset)
encoded_variants.shape

(2, 56, 40)

Verify that these encoded variants are the same as the encoded sequences from above.

In [8]:
np.all(encoded_variants == encoded_seqs)

True

Rather than specifying the wild-type sequence and offset manually, you can define them in `constants.py` and simply specify the dataset name `ds_name` when calling `enc.encode()`.

In [9]:
encoded_variants = enc.encode(encoding="one_hot,aa_index", variants=variants, ds_name="gb1")
encoded_variants.shape

(2, 56, 40)

We have added entries for all of our datasets in `constants.py`. You can also add your own dataset by matching the format we used for our datasets.